In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Configure OpenAI API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_version = os.getenv('OPENAI_API_VERSION')

# Initialize gpt-35-turbo and our embedding model
llm = AzureChatOpenAI(deployment_name="gptchat")
embeddings = OpenAIEmbeddings(deployment_id="embedding", chunk_size=1)

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME'),
                 azure_search_key=os.getenv('AZURE_COGNITIVE_SEARCH_API_KEY'),
                 index_name=os.getenv('AZURE_COGNITIVE_SEARCH_INDEX_NAME'),
                 embedding_function=embeddings.embed_query)


c:\Users\thinguy\AppData\Local\miniconda3\envs\qaenv\lib\site-packages\langchain\embeddings\openai.py:214: UserWarning: WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.
  warnings.warn(


In [2]:
# Load PDF files from "data" folder
directory = "./data"
def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
print(len(documents))

2


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

2190


In [14]:
docs[0]

Document(page_content='Digital – in the vehicle\n\nVehicle document wallet\n\nh c a b y a M s s a C - S\n\nl\n\nFamiliarize yourself with the contents of the Oper- ator\'s Manual directly via the vehicle\'s multimedia system (menu item "Vehi- cle information"). Start with the quick guide or broaden your knowledge with practical tips.\n\nHere you can find com- prehensive information about operating your vehicle and about serv- ices and guarantees in printed form.\n\nÉ22358425117ËÍ\n\n2235842511\n\nS-Class Maybach\n\nOperator\'s Manual\n\nOrder no. P223 0574 13 Part no. 223 584 25 11 Edition B-2023\n\nFront passenger air bag warning\n\n& WARNING Risk of injury or fatal injuries if the front passenger air bag is enabled\n\nIf the front passenger air bag is enabled, a child on the front passenger seat may be struck by the front passenger air bag in the event of an accident. # NEVER use a rearward-facing child\n\nrestraint system on a seat with an ENA- BLED FRONT AIR BAG. This can result in

In [4]:
# Add documents to Azure Search
acs.add_documents(documents=docs)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to 

['...',
 ...]

In [5]:
# Adapt if needed
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, 
                                                        rephrase the follow up question to be a standalone question. 

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=acs.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)


In [7]:
chat_history = []
query = "What are the airbag feature of the Mercedes-Benz S-Class Maybach as described in the manual?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])


Question: What are the airbag feature of the Mercedes-Benz S-Class Maybach as described in the manual?
Answer: The Mercedes-Benz S-Class Maybach Operator's Manual mentions that the front passenger airbag can cause injury or fatal injuries to a child if it is enabled during an accident. It also warns not to use a rearward-facing child restraint system on a seat with an enabled front airbag to avoid death or serious injury to the child. There is no other specific information about airbag features in the manual.


In [8]:
chat_history = [(query, result["answer"])]
query = "What is the fuel efficiency of the Maybach Mercedes car?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])


Question: What is the fuel efficiency of the Maybach Mercedes car?
Answer: The fuel efficiency of the Mercedes-Benz S-Class Maybach is not provided in the given context.


In [9]:
chat_history = [(query, result["answer"])]
query = "How can I turn S Maybach signal light on?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: How can I turn S Maybach signal light on?
Answer: To turn on the signal lights in a Mercedes-Maybach S-Class car, push the combination switch briefly to the point of resistance in the direction of arrow 2 or 4. The corresponding turn signal light will flash three times.


In [10]:
chat_history = [(query, result["answer"])]
query = "How can I turn S-Sedan signal light on?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: How can I turn S-Sedan signal light on?
Answer: To turn on the signal lights in a Mercedes-Maybach S-Sedan car, you need to push the combination switch briefly to the point of resistance in the direction of arrow 2 or 4. The corresponding turn signal light will flash three times.


In [11]:
chat_history = [(query, result["answer"])]
query = "In which situations, I can receive haptic feedback in s-sedan?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: In which situations, I can receive haptic feedback in s-sedan?
Answer: You can receive haptic feedback in a Mercedes-Maybach S-Sedan car in the following situations, for example: 
- When pressing a button on the user interface surface
- When scrolling in a list or table 
- When reaching a new area on the user interface surface, e.g. a pop-up window.


In [17]:
# Perform a vector similarity search
# Execute a pure vector similarity search
docs = acs.similarity_search(
    query="In which situations, I can receive haptic feedback in s-sedan?", 
    k=3,
    search_type="similarity")
print(docs[0].page_content)

The controls have touch-sensitive user interface surfaces. You can control these surfaces by pressing or swiping to adjust settings or to trigger functions, for example. In the area of the touchscreen, you will also receive haptic feedback in the form of a pulse or a vibration, or the surface structure will change on the touch-sensitive user interface surface, for example. You will receive haptic feedback in the following situations, for example: R When pressing a button on the user interface

surface

R When scrolling in a list or table R When reaching a new area on the user inter-

face surface, e.g. a pop-up window

When handling touch-sensitive user interface sur- faces, observe the following points to avoid prob- lems operating them: R Do not affix stickers or similar objects to the

surfaces

R Keep the surfaces protected from moisture R Keep the surfaces free of dust and dirt

(/ page 369).


In [19]:
# Perform a hybrid search
docs = acs.similarity_search(
    query="In which situations, I can receive haptic feedback in s-sedan?",
    k=3, 
    search_type="hybrid"
)
print(docs[0].page_content)

The controls have touch-sensitive user interface surfaces. You can control these surfaces by pressing or swiping to adjust settings or to trigger functions, for example. In the area of the touchscreen, you will also receive haptic feedback in the form of a pulse or a vibration, or the surface structure will change on the touch-sensitive user interface surface, for example. You will receive haptic feedback in the following situations, for example: R When pressing a button on the user interface

surface

R When scrolling in a list or table R When reaching a new area on the user inter-

face surface, e.g. a pop-up window

When handling touch-sensitive user interface sur- faces, observe the following points to avoid prob- lems operating them: R Do not affix stickers or similar objects to the

surfaces

R Keep the surfaces protected from moisture R Keep the surfaces free of dust and dirt

(/ page 369).
